# S3 폴더 -> 카탈로그 테이블 생성

In [10]:
import sys
print(sys.path)
print(sys.executable)

['c:\\Users\\USER\\aws_etl', 'c:\\Users\\USER\\anaconda3\\python311.zip', 'c:\\Users\\USER\\anaconda3\\DLLs', 'c:\\Users\\USER\\anaconda3\\Lib', 'c:\\Users\\USER\\anaconda3', '', 'c:\\Users\\USER\\anaconda3\\Lib\\site-packages', 'c:\\Users\\USER\\anaconda3\\Lib\\site-packages\\win32', 'c:\\Users\\USER\\anaconda3\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\USER\\anaconda3\\Lib\\site-packages\\Pythonwin']
c:\Users\USER\anaconda3\python.exe


In [11]:
import boto3
import re

# S3 클라이언트 생성
s3 = boto3.client("s3")

bucket_name = "schoolfriends-kmj"

# 버킷 내 모든 객체 가져오기
response = s3.list_objects_v2(Bucket=bucket_name, Delimiter="/")
print(response['CommonPrefixes'])
for folder in response.get('CommonPrefixes', []):
    print(folder['Prefix'])

[{'Prefix': 'at_basic/'}, {'Prefix': 'at_criterion/'}, {'Prefix': 'at_enable/'}, {'Prefix': 'at_explanation/'}, {'Prefix': 'at_file/'}, {'Prefix': 'at_formula/'}, {'Prefix': 'at_inspection/'}, {'Prefix': 'at_low/'}, {'Prefix': 'at_psy/'}, {'Prefix': 'at_question/'}, {'Prefix': 'at_scale/'}, {'Prefix': 'at_sub/'}, {'Prefix': 'at_synthetic/'}, {'Prefix': 'at_upper/'}, {'Prefix': 'at_user/'}, {'Prefix': 'common_cd/'}, {'Prefix': 'common_group/'}, {'Prefix': 'kko_log/'}, {'Prefix': 'kko_send/'}, {'Prefix': 'member_add/'}, {'Prefix': 'member_auth/'}, {'Prefix': 'member_sign/'}, {'Prefix': 'psy_class/'}, {'Prefix': 'psy_estimate/'}, {'Prefix': 'psy_item/'}, {'Prefix': 'psy_ord/'}, {'Prefix': 'psy_target/'}, {'Prefix': 'school_class/'}, {'Prefix': 'school_info/'}, {'Prefix': 'school_student/'}, {'Prefix': 'school_teacher/'}, {'Prefix': 'ui_image/'}]
at_basic/
at_criterion/
at_enable/
at_explanation/
at_file/
at_formula/
at_inspection/
at_low/
at_psy/
at_question/
at_scale/
at_sub/
at_syntheti

In [ ]:
# CSV 파일만 폴더 만들어서 이동 및 삭제 처리
for obj in response.get("Contents", []): # 'Contents' 키값 가지고 있는 값 (무슨 형태인지를 모르겠네)
    key = obj["Key"]
    if "/" not in key and key.lower().endswith(".csv"):
        print(f"Processing file: {key}")
        match = re.findall(r"[A-Za-z]+", key) # 객체 중 알파벳_알파벳 패턴 찾기
        print(match)
        if not match:
            print(f"No matching pattern found in {key}, skipping.")
            continue
        folder_name = "_".join(match).lower() # 패턴을 폴더 이름으로 정의하고
        print(folder_name)
        new_key = f"{folder_name}/{key}" # 새로 정의한 폴더 이름/기존 파일 이름
        print(f"Moving {key} to {new_key}")

        # S3 내에서 파일 복사
        s3.copy_object(
            Bucket=bucket_name,
            CopySource={"Bucket": bucket_name, "Key": key},
            Key=new_key
        )

        # 원본 삭제
        s3.delete_object(Bucket=bucket_name, Key=key)
    else:
        pass

Processing file: AT_BASIC_INQUIRY_VAL_202508291453.csv
<re.Match object; span=(0, 8), match='AT_BASIC'>
at_basic
Moving AT_BASIC_INQUIRY_VAL_202508291453.csv to at_basic/AT_BASIC_INQUIRY_VAL_202508291453.csv
Processing file: AT_FORMULA_CODE_202508291455.csv
<re.Match object; span=(0, 10), match='AT_FORMULA'>
at_formula
Moving AT_FORMULA_CODE_202508291455.csv to at_formula/AT_FORMULA_CODE_202508291455.csv
Processing file: AT_LOW_SCALE_202508291455.csv
<re.Match object; span=(0, 6), match='AT_LOW'>
at_low
Moving AT_LOW_SCALE_202508291455.csv to at_low/AT_LOW_SCALE_202508291455.csv
Processing file: AT_QUESTION_202508291458.csv
<re.Match object; span=(0, 11), match='AT_QUESTION'>
at_question
Moving AT_QUESTION_202508291458.csv to at_question/AT_QUESTION_202508291458.csv
Processing file: AT_QUESTION_CHOICE_202508291459.csv
<re.Match object; span=(0, 11), match='AT_QUESTION'>
at_question
Moving AT_QUESTION_CHOICE_202508291459.csv to at_question/AT_QUESTION_CHOICE_202508291459.csv
Processing 

AttributeError: 'NoneType' object has no attribute 'group'

# S3 폴더별 크롤러 만들때

In [ ]:
# 폴더 이름 리스트화
s3 = boto3.client("s3")
bucket_name = "schoolfriends-bym"
response = s3.list_objects_v2(Bucket=bucket_name, Delimiter="/")
folder_names = []
for folder in response.get("CommonPrefixes", []):
    folder_name = folder["Prefix"].split("/")[0]
    folder_names.append(folder_name) # 버킷안 폴더이름 추출

print('s3에 있는 폴더명 :', folder_names)

# s3 폴더 별로 크롤러 만들어서 아테네 테이블 만들때
glue = boto3.client("glue", region_name="ap-northeast-2")

# 공통 파라미터
role_name = 'arn:aws:iam::932744610695:role/service-role/AWSGlueServiceRole-Schoolfriends' # Glue 서비스 역할 ARN
database_name = 'schoolfriends' # 크롤링해 만들 데이터베이스 이름
bucket_name = 'schoolfriends-bym' # 크롤러할 폴더가 있는 버킷 이름

for folder in folder_names:
    crawler_name = f"crawler_{folder}"
    s3_target_path = f"s3://{bucket_name}/{folder}/"
    
    try:
        response = glue.create_crawler(
            Name=crawler_name,
            Role=role_name,
            DatabaseName=database_name,
            Targets={'S3Targets': [{'Path': s3_target_path}]},
            SchemaChangePolicy={
                'UpdateBehavior': 'LOG', # 스키마가 바뀐경우 log=로그만 남기고, 카탈로그 정의는 그대로 둠/'UPDATE_IN_DATABASE'=카탈로그 정의를 바꿔줌
                'DeleteBehavior': 'LOG' # 삭제된 데이터에 대해 log=로그만 남기고, 카탈로그 정의는 그대로 둠/'DEPRECATE_IN_DATABASE'=카탈로그 정의에서 삭제된 파티션을 제거
            } # 크롤러는 전체 데이터를 스캔할 수도, 새롭게 추가된 데이터만 스캔할 수도 있음 / 그러나 기본적으로 S3의 전체 경로를 스캔함 -> 그렇기 때문에 파티션 삭제만 인지하고, 파일삭제는 인지하지 못함.
        )
        print(f"Crawler '{crawler_name}' created successfully.")
    except glue.exceptions.AlreadyExistsException:
        print(f"Crawler '{crawler_name}' already exists.")
    except Exception as e:
        print(f"Error creating crawler '{crawler_name}': {e}")

s3에 있는 폴더명 : ['at_basic_inquiry_val', 'at_formula_code', 'at_low_scale', 'at_question', 'at_question_choice', 'at_question_extra', 'at_scale_code', 'at_sub_psy_item', 'at_user_formula_score', 'at_user_scale_score', 'at_user_testing_paper', 'at_user_testing_paper_pn', 'at_user_testing_paper_sct', 'at_user_testing_paper_tr', 'athena', 'member', 'psy_class', 'psy_estimate', 'psy_ord', 'psy_ord_class', 'psy_target', 'school_class', 'school_info', 'school_student', 'school_teacher']
Crawler 'crawler_at_basic_inquiry_val' already exists.
Crawler 'crawler_at_formula_code' already exists.
Crawler 'crawler_at_low_scale' already exists.
Crawler 'crawler_at_question' already exists.
Crawler 'crawler_at_question_choice' already exists.
Crawler 'crawler_at_question_extra' already exists.
Crawler 'crawler_at_scale_code' already exists.
Crawler 'crawler_at_sub_psy_item' already exists.
Crawler 'crawler_at_user_formula_score' already exists.
Crawler 'crawler_at_user_scale_score' already exists.
Crawle

# 크롤러 속성 변경하고 싶을 때

In [25]:
# 각 크롤러의 속성을 변경하고 싶은 경우
# - 이미 생성된 Glue의 크롤러의 속성을 변경하고 싶은 경우
# - 예를 들어 스키마 구조 변경 정책(SchemaChangePolicy) 등 일괄 변경
import boto3

glue = boto3.client("glue", region_name="ap-northeast-2")

# 크롤러 이름 리스트 가져오기
response = glue.list_crawlers()
crawler_names = response.get("CrawlerNames", [])
print(crawler_names)

['amazon_reviews_partitioned_crawler', 'crawler_at_basic_inquiry_val', 'crawler_at_formula_code', 'crawler_at_low_scale', 'crawler_at_question', 'crawler_at_question_choice', 'crawler_at_question_extra', 'crawler_at_scale_code', 'crawler_at_sub_psy_item', 'crawler_at_user_formula_score', 'crawler_at_user_scale_score', 'crawler_at_user_testing_paper', 'crawler_at_user_testing_paper_pn', 'crawler_at_user_testing_paper_sct', 'crawler_at_user_testing_paper_tr', 'crawler_member', 'crawler_psy_class', 'crawler_psy_estimate', 'crawler_psy_ord', 'crawler_psy_target', 'crawler_school_class', 'crawler_school_info', 'crawler_school_student', 'crawler_school_teacher', 'schoolfriends_member_crawler']


In [35]:
import time
for crawler in crawler_names:
    if crawler.startswith("crawler_"):
        try:
            glue.update_crawler(
                Name=crawler,
                SchemaChangePolicy={
                    "UpdateBehavior": "LOG",
                    "DeleteBehavior": "LOG"
                }
            )
            print(f"✅ {crawler} 의 SchemaChangePolicy 업데이트 완료")
            time.sleep(1)
        except Exception as e:
            print(f"❌ {crawler} 업데이트 실패: {repr(e)}")

✅ crawler_at_basic_inquiry_val 의 SchemaChangePolicy 업데이트 완료
✅ crawler_at_formula_code 의 SchemaChangePolicy 업데이트 완료
✅ crawler_at_low_scale 의 SchemaChangePolicy 업데이트 완료
✅ crawler_at_question 의 SchemaChangePolicy 업데이트 완료
✅ crawler_at_question_choice 의 SchemaChangePolicy 업데이트 완료
✅ crawler_at_question_extra 의 SchemaChangePolicy 업데이트 완료
✅ crawler_at_scale_code 의 SchemaChangePolicy 업데이트 완료
✅ crawler_at_sub_psy_item 의 SchemaChangePolicy 업데이트 완료
✅ crawler_at_user_formula_score 의 SchemaChangePolicy 업데이트 완료
✅ crawler_at_user_scale_score 의 SchemaChangePolicy 업데이트 완료
✅ crawler_at_user_testing_paper 의 SchemaChangePolicy 업데이트 완료
✅ crawler_at_user_testing_paper_pn 의 SchemaChangePolicy 업데이트 완료
✅ crawler_at_user_testing_paper_sct 의 SchemaChangePolicy 업데이트 완료
✅ crawler_at_user_testing_paper_tr 의 SchemaChangePolicy 업데이트 완료
✅ crawler_member 의 SchemaChangePolicy 업데이트 완료
✅ crawler_school_class 의 SchemaChangePolicy 업데이트 완료


# 크롤러 생성일자 기준 조회 및 삭제

In [ ]:
import boto3
from datetime import datetime, timedelta, timezone

glue = boto3.client("glue")

crawlers = []
next_token = None

while True:
    if next_token:
        resp = glue.get_crawlers(NextToken=next_token)
    else:
        resp = glue.get_crawlers()

    crawlers.extend(resp.get("Crawlers", []))
    next_token = resp.get("NextToken")

    if not next_token:
        break

print(f"Total crawlers found: {len(crawlers)}")

cutoff = datetime.now(timezone.utc) - timedelta(days=30)
recent_create_crawlers = []
for obj in crawlers:
    creation_time = obj.get("CreationTime")
    if creation_time and creation_time >= cutoff:
        recent_create_crawlers.append(obj)

for c in recent_create_crawlers:
    print(c["Name"], c["CreationTime"])


Total crawlers found: 54
crawler_at_basic 2025-11-14 09:31:07+09:00
crawler_at_criterion 2025-11-14 09:31:08+09:00
crawler_at_enable 2025-11-14 09:31:08+09:00
crawler_at_explanation 2025-11-14 09:31:08+09:00
crawler_at_file 2025-11-14 09:31:08+09:00
crawler_at_formula 2025-11-14 09:31:14+09:00
crawler_at_inspection 2025-11-14 09:31:14+09:00
crawler_at_low 2025-11-14 09:31:14+09:00
crawler_at_psy 2025-11-14 09:31:14+09:00
crawler_at_scale 2025-11-14 09:31:15+09:00
crawler_at_sub 2025-11-14 09:31:15+09:00
crawler_at_synthetic 2025-11-14 09:31:22+09:00
crawler_at_upper 2025-11-14 09:31:22+09:00
crawler_at_user 2025-11-14 09:31:22+09:00
crawler_common_cd 2025-11-14 09:31:23+09:00
crawler_common_group 2025-11-14 09:31:23+09:00
crawler_kko_log 2025-11-14 09:31:24+09:00
crawler_kko_send 2025-11-14 09:31:24+09:00
crawler_member_add 2025-11-14 09:31:26+09:00
crawler_member_auth 2025-11-14 09:31:26+09:00
crawler_member_sign 2025-11-14 09:31:26+09:00
crawler_psy_item 2025-11-14 09:31:31+09:00
cra

In [ ]:
recent_create_crawlers

[{'Name': 'crawler_at_basic',
  'Role': 'service-role/AWSGlueServiceRole-Schoolfriends',
  'Targets': {'S3Targets': [{'Path': 's3://schoolfriends-bym/at_basic/',
     'Exclusions': []}],
   'JdbcTargets': [],
   'MongoDBTargets': [],
   'DynamoDBTargets': [],
   'CatalogTargets': [],
   'DeltaTargets': [],
   'IcebergTargets': [],
   'HudiTargets': []},
  'DatabaseName': 'schoolfriends',
  'Classifiers': [],
  'RecrawlPolicy': {'RecrawlBehavior': 'CRAWL_EVERYTHING'},
  'SchemaChangePolicy': {'UpdateBehavior': 'LOG', 'DeleteBehavior': 'LOG'},
  'LineageConfiguration': {'CrawlerLineageSettings': 'DISABLE'},
  'State': 'READY',
  'CrawlElapsedTime': 0,
  'CreationTime': datetime.datetime(2025, 11, 14, 9, 31, 7, tzinfo=tzlocal()),
  'LastUpdated': datetime.datetime(2025, 11, 14, 9, 31, 7, tzinfo=tzlocal()),
  'Version': 1,
  'LakeFormationConfiguration': {'UseLakeFormationCredentials': False,
   'AccountId': ''}},
 {'Name': 'crawler_at_criterion',
  'Role': 'service-role/AWSGlueServiceRole

In [ ]:
for c in recent_create_crawlers:
    crawler_name = c["Name"]
    created = c["CreationTime"]
    print(f"Deleting: {crawler_name} (Created: {created})")

    glue.delete_crawler(Name=crawler_name)

print("=== Delete Completed ===")

Deleting: crawler_at_basic (Created: 2025-11-14 09:31:07+09:00)
Deleting: crawler_at_criterion (Created: 2025-11-14 09:31:08+09:00)
Deleting: crawler_at_enable (Created: 2025-11-14 09:31:08+09:00)
Deleting: crawler_at_explanation (Created: 2025-11-14 09:31:08+09:00)
Deleting: crawler_at_file (Created: 2025-11-14 09:31:08+09:00)
Deleting: crawler_at_formula (Created: 2025-11-14 09:31:14+09:00)
Deleting: crawler_at_inspection (Created: 2025-11-14 09:31:14+09:00)
Deleting: crawler_at_low (Created: 2025-11-14 09:31:14+09:00)
Deleting: crawler_at_psy (Created: 2025-11-14 09:31:14+09:00)
Deleting: crawler_at_scale (Created: 2025-11-14 09:31:15+09:00)
Deleting: crawler_at_sub (Created: 2025-11-14 09:31:15+09:00)
Deleting: crawler_at_synthetic (Created: 2025-11-14 09:31:22+09:00)
Deleting: crawler_at_upper (Created: 2025-11-14 09:31:22+09:00)
Deleting: crawler_at_user (Created: 2025-11-14 09:31:22+09:00)
Deleting: crawler_common_cd (Created: 2025-11-14 09:31:23+09:00)
Deleting: crawler_common_

# 데이터 카탈로그 스키마를 변경하고 싶을 때

In [3]:
# 크롤링 후 테이블 데이터 타입 재정의
# 강제 string
import boto3

region_name = "ap-northeast-2"
database_name = 'schoolfriends'

# 클라이언트 생성
glue_client = boto3.client("glue", region_name=region_name)
table_list = []
table_paginator = glue_client.get_paginator('get_tables') # get_paginator은 api은 최대 100개 반환하기 때문에 이를 방지하기 위해 페이지네이션 처리
for page in table_paginator.paginate(DatabaseName=database_name): # 데이터베이스 이름이 database_name인 페이지의 순환 설정
    print(page['TableList']) # 각 페이지의 테이블 목록 출력
    for table in page['TableList']: # 각 테이블에 대한 순환 설정
        print(table['Name'])
        table_list.append(table['Name']) # 테이블 이름을 리스트에 추가

for table_name in table_list: # 모든 테이블 이름 반복
    print("----------순환 중인 테이블----------")
    print('테이블 이름', table_name) # 순환 중인 테이블 이름
    table_info = glue_client.get_table(DatabaseName=database_name, Name=table_name)["Table"] # 순환중인 테이블 정보 가져오기
    table_col_info = table_info['StorageDescriptor'] # 테이블의 StorageDescriptor 정보 가져오기 ex {'Columns': [{'Name': 'user_testing_no', 'Type': 'string'}, {'Name': 'psy_item_id', 'Type': 'string'},
    print("----------테이블 정보----------")
    print(table_col_info)
    for col in table_col_info['Columns']:
        print("------------변경되는 컬럼의 타입------------")
        print(col['Name'], col['Type'])
        col['Type'] = 'string'  # 모든 컬럼의 타입을 string으로 변경
    print("-----------------------------------------------------")
    print('Name :', table_info["Name"])
    print('Description :', table_info.get("Description"))
    print('Owner :', table_info.get("Owner"))
    print('Parameters :', table_info.get("Parameters", {}))
    print('TableType :', table_info.get("TableType", "EXTERNAL_TABLE"))
    print('PartitionKeys :', table_info.get("PartitionKeys", []))
    print('StorageDescriptor :', table_info)
    print("-----------------------------------------------------")

    # 테이블 정보 업데이트
    glue_client.update_table(
        DatabaseName=database_name,
        TableInput={
            'Name': table_info["Name"], # 테이블 이름
            'Description': table_info.get("Description", ""), # 테이블 설명
            'Owner': table_info.get("Owner"), # 테이블 소유자
            'Parameters': table_info.get("Parameters", {}), # 테이블 파라미터
            'TableType': table_info.get("TableType", "EXTERNAL_TABLE"), # 테이블 타입
            'PartitionKeys': table_info.get("PartitionKeys", []), # 파티션 키
            'StorageDescriptor': table_col_info
        }
    )
    print(f"✅ {table_name} 의 모든 컬럼 타입이 string으로 변경 완료")

[{'Name': 'at_basic_inquiry_val', 'DatabaseName': 'schoolfriends', 'Description': '', 'Owner': 'owner', 'CreateTime': datetime.datetime(2025, 9, 1, 14, 49, 45, tzinfo=tzlocal()), 'UpdateTime': datetime.datetime(2025, 9, 3, 19, 6, 42, tzinfo=tzlocal()), 'Retention': 0, 'StorageDescriptor': {'Columns': [{'Name': 'user_testing_no', 'Type': 'string'}, {'Name': 'psy_item_id', 'Type': 'string'}, {'Name': 'psy_item_ver', 'Type': 'string'}, {'Name': 'sub_psy_item_id', 'Type': 'string'}, {'Name': 'age_ext_sub_psy_item_id', 'Type': 'string'}, {'Name': 'kabc_sub_psy_item_id', 'Type': 'string'}, {'Name': 'testing_org', 'Type': 'string'}, {'Name': 'birthday', 'Type': 'string'}, {'Name': 'sex', 'Type': 'string'}, {'Name': 'at_region_cd', 'Type': 'string'}, {'Name': 'testing_date', 'Type': 'string'}, {'Name': 'name', 'Type': 'string'}, {'Name': 'at_age_cd', 'Type': 'string'}, {'Name': 'at_extension_age_cd', 'Type': 'string'}, {'Name': 'at_school_type_cd', 'Type': 'string'}, {'Name': 'at_school_age_cd

# 데이터 카탈로그 컬럼명 변경하고 싶을 때

In [5]:
# 크롤링이 잘못되어 컬럼명을 다시 정의해야하는 경우
import boto3

region_name = "ap-northeast-2"
database_name = 'schoolfriends'

# 클라이언트 생성 후 테이블 확인
glue_client = boto3.client("glue", region_name=region_name)
table_list = []
table_paginator = glue_client.get_paginator('get_tables') # get_paginator은 api은 최대 100개 반환하기 때문에 이를 방지하기 위해 페이지네이션 처리
for page in table_paginator.paginate(DatabaseName=database_name): # 데이터베이스 이름이 database_name인 페이지의 순환 설정
    print(page['TableList']) # 각 페이지의 테이블 목록 출력
    for table in page['TableList']: # 각 테이블에 대한 순환 설정
        print(table['Name'])
        table_list.append(table['Name']) # 테이블 이름을 리스트에 추가
        print(table_list) # 무슨 테이블이 존재하는지 확인

# 특정 테이블 선택해서 컬럼명 수정
target_table_name = 'school_teacher'
table_info = glue_client.get_table(DatabaseName=database_name, Name=target_table_name)["Table"] # 선택한 테이블 정보 가져오기
table_col_info = table_info['StorageDescriptor'] # 테이블의 StorageDescriptor 정보 가져오기 ex {'Columns': [{'Name': 'user_testing_no', 'Type': 'string'}, {'Name': 'psy_item_id', 'Type': 'string'},
modify_column_name_mapping = ['school_code', 'member_no', 'use_yn', 'memo', 'reg_member_no', 'reg_date', 'mod_member_no', 'mod_date']
for column_name, col_info in zip(modify_column_name_mapping, table_col_info['Columns']):
    print("- 변경되는 컬럼의 정보 ------------")
    print(col_info['Name'], col_info['Type'])
    col_info['Name'] = column_name  # 모든 컬럼의 이름을 변경
    print(f'컬럼명: {col_info["Name"]}으로 변경 완료')
print(table_info['StorageDescriptor'])

[{'Name': 'at_basic_inquiry_val', 'DatabaseName': 'schoolfriends', 'Description': '', 'Owner': 'owner', 'CreateTime': datetime.datetime(2025, 9, 1, 14, 49, 45, tzinfo=tzlocal()), 'UpdateTime': datetime.datetime(2025, 9, 4, 10, 2, 5, tzinfo=tzlocal()), 'Retention': 0, 'StorageDescriptor': {'Columns': [{'Name': 'user_testing_no', 'Type': 'string'}, {'Name': 'psy_item_id', 'Type': 'string'}, {'Name': 'psy_item_ver', 'Type': 'string'}, {'Name': 'sub_psy_item_id', 'Type': 'string'}, {'Name': 'age_ext_sub_psy_item_id', 'Type': 'string'}, {'Name': 'kabc_sub_psy_item_id', 'Type': 'string'}, {'Name': 'testing_org', 'Type': 'string'}, {'Name': 'birthday', 'Type': 'string'}, {'Name': 'sex', 'Type': 'string'}, {'Name': 'at_region_cd', 'Type': 'string'}, {'Name': 'testing_date', 'Type': 'string'}, {'Name': 'name', 'Type': 'string'}, {'Name': 'at_age_cd', 'Type': 'string'}, {'Name': 'at_extension_age_cd', 'Type': 'string'}, {'Name': 'at_school_type_cd', 'Type': 'string'}, {'Name': 'at_school_age_cd'

In [6]:
table_col_info

{'Columns': [{'Name': 'school_code', 'Type': 'string'},
  {'Name': 'member_no', 'Type': 'string'},
  {'Name': 'use_yn', 'Type': 'string'},
  {'Name': 'memo', 'Type': 'string'},
  {'Name': 'reg_member_no', 'Type': 'string'},
  {'Name': 'reg_date', 'Type': 'string'},
  {'Name': 'mod_member_no', 'Type': 'string'},
  {'Name': 'mod_date', 'Type': 'string'}],
 'Location': 's3://schoolfriends-bym/school_teacher/',
 'InputFormat': 'org.apache.hadoop.mapred.TextInputFormat',
 'OutputFormat': 'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat',
 'Compressed': False,
 'NumberOfBuckets': -1,
 'SerdeInfo': {'SerializationLibrary': 'org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe',
  'Parameters': {'field.delim': ','}},
 'BucketColumns': [],
 'SortColumns': [],
 'Parameters': {'sizeKey': '5321',
  'objectCount': '1',
  'UPDATED_BY_CRAWLER': 'crawler_school_teacher',
  'recordCount': '61',
  'CrawlerSchemaSerializerVersion': '1.0',
  'averageRecordSize': '87',
  'compressionType': 'none',

In [ ]:
# 테이블 정보 업데이트
glue_client.update_table(
    DatabaseName=database_name,
    TableInput={
        'Name': target_table_name, # 수정해야 하는 테이블 이름
        'Description': table_info.get("Description", ""), # 테이블 설명
        'Owner': table_info.get("Owner"), # 테이블 소유자
        'Parameters': table_info.get("Parameters", {}), # 테이블 파라미터
        'TableType': table_info.get("TableType", "EXTERNAL_TABLE"), # 테이블 타입
        'PartitionKeys': table_info.get("PartitionKeys", []), # 파티션 키
        'StorageDescriptor': table_info['StorageDescriptor'] # 여기서도 참조 원리가 반영되어 table_col_info이 아닌 table_info['StorageDescriptor']로 해야함
    }
)
print(f"✅ {target_table_name} 의 모든 컬럼 이름 변경 완료")

✅ school_teacher 의 모든 컬럼 이름 변경 완료


# glue job 만들어야 할때

## 1. rds에 glue data catalog 테이블과 동일한 테이블 생성 과정

In [1]:
# data catalog 테이블명 읽어서 rds에 동일한 테이블명으로 테이블 생성
import boto3

# AWS 리전과 데이터베이스 이름 지정
region_name = "ap-northeast-2"
database_name = "schoolfriends"

# Glue 클라이언트 생성
glue_client = boto3.client("glue", region_name=region_name)

# Glue Data Catalog에서 테이블명 가져오기
table_list = []
paginator = glue_client.get_paginator("get_tables")

for page in paginator.paginate(DatabaseName=database_name):
    for table in page["TableList"]:
        table_name = table["Name"]
        table_list.append(table_name)

print("📌 Glue Data Catalog에 등록된 테이블 목록:")
for t in table_list:
    print("-", t)

📌 Glue Data Catalog에 등록된 테이블 목록:
- aggression_norms
- aggression_point
- at_basic_inquiry_val
- at_formula_code
- at_low_scale
- at_question
- at_question_choice
- at_question_extra
- at_scale_code
- at_sub_psy_item
- at_user_formula_score
- at_user_scale_score
- at_user_testing_paper
- at_user_testing_paper_pn
- at_user_testing_paper_sct
- at_user_testing_paper_tr
- member
- prosociality_norms
- prosociality_point
- psy_class
- psy_estimate
- psy_ord
- psy_ord_class
- psy_target
- school_class
- school_info
- school_student
- school_teacher


In [13]:
# RDS에 Glue Data Catalog 테이블과 동일한 테이블 생성
import pymysql

# RDS 연결 설정 (ec2 점프 서버 활용해 접속)
conn = pymysql.connect(
    host="127.0.0.1",
    port=3307,
    user="db_rnd_admin",
    password="0dlstk12#",
    database="schoolfriend_rnd",
    charset="utf8mb4"
)

cursor = conn.cursor()

def glue_type_to_mysql(glue_type: str) -> str: # Glue Data Catalog의 타입은 Hive/Parquet 계열이라 MySQL 타입으로 바꿔줘야 합니다.
    mapping = {
        "string": "VARCHAR(255)",
        "bigint": "BIGINT",
        "int": "INT",
        "double": "DOUBLE",
        "float": "FLOAT",
        "boolean": "BOOLEAN",
        "timestamp": "DATETIME"
    }
    return mapping.get(glue_type.lower(), "VARCHAR(255)")

for table_name in table_list:
    # Glue 테이블 스키마 가져오기
    table_info = glue_client.get_table(DatabaseName=database_name, Name=table_name)
    columns = table_info["Table"]["StorageDescriptor"]["Columns"]
    partition_keys = table_info["Table"].get("PartitionKeys", [])
    
    # 컬럼 SQL 정의
    column_defs = []
    for col in columns + partition_keys:
        col_name = col["Name"]
        col_type = glue_type_to_mysql(col["Type"])
        column_defs.append(f"`{col_name}` {col_type}")
    
    # CREATE TABLE 쿼리
    create_sql = f"""
    CREATE TABLE IF NOT EXISTS `{table_name}` (
        {", ".join(column_defs)}
    );
    """
    print(create_sql)  # 👉 확인용 출력
    cursor.execute(create_sql)

conn.commit()
cursor.close()
conn.close()



    CREATE TABLE IF NOT EXISTS `aggression_norms` (
        `psy_code` VARCHAR(255), `top5_mean` DOUBLE, `total_mean` DOUBLE, `weighted_mean` DOUBLE, `reg_date` VARCHAR(255), `purpose` VARCHAR(255)
    );
    

    CREATE TABLE IF NOT EXISTS `aggression_point` (
        `student_code` VARCHAR(255), `num_point_agg` BIGINT, `num_point_14no` BIGINT, `agg*14no_num_point` BIGINT, `psy_code` VARCHAR(255)
    );
    

    CREATE TABLE IF NOT EXISTS `at_basic_inquiry_val` (
        `user_testing_no` VARCHAR(255), `psy_item_id` VARCHAR(255), `psy_item_ver` VARCHAR(255), `sub_psy_item_id` VARCHAR(255), `age_ext_sub_psy_item_id` VARCHAR(255), `kabc_sub_psy_item_id` VARCHAR(255), `testing_org` VARCHAR(255), `birthday` VARCHAR(255), `sex` VARCHAR(255), `at_region_cd` VARCHAR(255), `testing_date` VARCHAR(255), `name` VARCHAR(255), `at_age_cd` VARCHAR(255), `at_extension_age_cd` VARCHAR(255), `at_school_type_cd` VARCHAR(255), `at_school_age_cd` VARCHAR(255), `subs_yn` VARCHAR(255), `supp_yn` VARCHAR

## 2. Glue Job 생성해서 S3, Data Catalog 데이터 추출하여 RDS에 적재

In [4]:
# rds 접속해 테이블 목록 가지고 오기
# RDS에 Glue Data Catalog 테이블과 동일한 테이블 생성
import pymysql

# RDS 연결 설정 (ec2 점프 서버 활용해 접속)
conn = pymysql.connect(
    host="127.0.0.1",
    port=3307,
    user="db_rnd_admin",
    password="0dlstk12#",
    database="schoolfriend_rnd",
    charset="utf8mb4"
)

cursor = conn.cursor()
table_list = []
cursor.execute("SHOW TABLES;")
for (table_name,) in cursor.fetchall():
    table_list.append(table_name)
print(table_list)

conn.commit()
cursor.close()
conn.close()


['aggression_norms', 'aggression_point', 'at_basic_inquiry_val', 'at_formula_code', 'at_low_scale', 'at_question', 'at_question_choice', 'at_question_extra', 'at_scale_code', 'at_sub_psy_item', 'at_user_formula_score', 'at_user_scale_score', 'at_user_testing_paper', 'at_user_testing_paper_pn', 'at_user_testing_paper_sct', 'at_user_testing_paper_tr', 'member', 'prosociality_norms', 'prosociality_point', 'psy_class', 'psy_estimate', 'psy_ord', 'psy_ord_class', 'psy_target', 'school_class', 'school_info', 'school_student', 'school_teacher']


In [ ]:
import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job

args = getResolvedOptions(sys.argv, ['JOB_NAME'])

sc = SparkContext()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)
job.init(args['JOB_NAME'], args)

table_list = ['aggression_norms', 'aggression_point', 'at_basic_inquiry_val', 'at_formula_code', 'at_low_scale', 'at_question', 'at_question_choice', 'at_question_extra', 'at_scale_code', 'at_sub_psy_item', 'at_user_formula_score', 'at_user_scale_score', 'at_user_testing_paper', 'at_user_testing_paper_pn', 'at_user_testing_paper_sct', 'at_user_testing_paper_tr', 'member', 'prosociality_norms', 'prosociality_point', 'psy_class', 'psy_estimate', 'psy_ord', 'psy_ord_class', 'psy_target', 'school_class', 'school_info', 'school_student', 'school_teacher']

for table in table_list:
    print(f'Processing table: {table}')

    # 1️⃣ Glue Data Catalog에서 S3 데이터를 읽어오기
    source_dyf = glueContext.create_dynamic_frame.from_catalog(
        database="schoolfriends",      # 크롤러가 만든 Glue 데이터베이스(카탈로그 DB 이름)
        table_name="school_class"       # 크롤러가 만든 테이블(카탈로그 테이블 이름)
    )
    # ⮕ Glue Data Catalog에 등록된 'schoolfriends.school_info'의 스키마 + S3 경로를 이용해
    #    S3 파일을 DynamicFrame으로 로드.
    #    - DynamicFrame은 Glue가 제공하는 스키마 유연(느슨) 타입 컨테이너.
    #    - 스키마 불일치나 중첩/널 처리에 강함(ResolveChoice, ApplyMapping 등과 궁합).


    # 2️⃣ RDS(MySQL)에 적재할 옵션 지정
    connection_options = {
        "dbtable": "school_class",           # RDS에서 쓸 테이블명(이미 생성해 둔 그 이름)
        "database": "schoolfriend_rnd",     # RDS의 DB 스키마명
        "connectionName": "glue-to-rds-connection"  # (참고) 보통 필요 없음. 아래 catalog_connection으로 지정함.
    }
    # ⮕ from_jdbc_conf에 넘길 옵션.
    #    핵심은 dbtable(타겟 테이블)과 database(RDS DB 이름).
    #    ※ 주의: 실제 JDBC 네트워크/자격 정보는 아래 catalog_connection에 담긴 "Glue Connection" 리소스를 사용.


    # 3️⃣ RDS(MySQL)로 데이터 쓰기
    glueContext.write_dynamic_frame.from_jdbc_conf(
        frame=source_dyf,
        catalog_connection="glue-to-rds-connection",  # Glue 콘솔 > Data Catalog > Connections 에서 만든 연결 이름
        connection_options=connection_options
    )
    # ⮕ DynamicFrame → RDS로 적재.
    #    - catalog_connection: Glue Connection 리소스 이름(여기에 JDBC URL, VPC/서브넷/보안그룹, 사용자/암호(또는 시크릿) 설정).
    #    - connection_options: dbtable/database 등 테이블 타깃 지정.
    #    - 옵션 확장 예: {"preactions":"TRUNCATE TABLE school_info","batchsize":"5000"} 등.

# 4️⃣ Job 종료
job.commit()
# ⮕ 잡을 정상 종료로 마무리(커밋). 이게 있어야 Glue 콘솔에서 "Succeeded"로 기록됨.